<a href="https://colab.research.google.com/github/larissabooth/cv_project/blob/main/CV_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/Separius/SimCLRv2-Pytorch.git "/content/drive/My Drive/computer vision project/simclr_pytorch"

Cloning into '/content/drive/My Drive/computer vision project/simclr_pytorch'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 20 (delta 5), reused 4 (delta 4), pack-reused 11
Unpacking objects: 100% (20/20), 21.54 KiB | 84.00 KiB/s, done.


In [ ]:
!pip install Pillow
!pip install -U image
!pip install opencv-python
!pip install torch
!pip install torchvision

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
# You might not have tqdm, which gives you nice progress bars
!pip install tqdm
from tqdm.notebook import tqdm
import os
import copy
import pandas as pd
import PIL 
import random
from collections import defaultdict

# For reproducibility
torch.manual_seed(1234)

# CPU / GPU
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda:0')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
from tqdm import tqdm

#FROM PSET 5:

# else:
#   data_dir = "./data"

# downloads_dir = data_dir + '/downloads'
# datasets_dir = data_dir + '/datasets'
# models_dir = data_dir + '/models'
# pretrained_models = data_dir + '/pretrained_models'
# q2c_data_dir = data_dir + '/q2c_data'

# os.makedirs(downloads_dir, exist_ok=True)
# os.makedirs(datasets_dir, exist_ok=True)
# os.makedirs(models_dir, exist_ok=True)
# os.makedirs(pretrained_models, exist_ok=True)
# os.makedirs(q2c_data_dir, exist_ok=True)

# mias_dataset_dir = datasets_dir + '/mias_dataset'
# external_dataset_dir = datasets_dir + '/external_dataset'

In [ ]:
# # Get ImageNet 2012 validation labels to verify correct conversion to pytorch

# import urllib.request

# # URL for the validation set images and labels
# # url_images = 'http://www.image-net.org/challenges/LSVRC/2012/dd31405981ef5f776aa17412e1f0c112/ILSVRC2012_img_val.tar'
# url_labels = 'https://raw.githubusercontent.com/fh295/semanticCNN/master/imagenet_labels/ILSVRC2012_validation_ground_truth.txt'

# # Directory to save the validation set
# val_path = './val/'

# # Create the directory if it doesn't exist
# if not os.path.exists(val_path):
#     os.makedirs(val_path)

# # # Download the validation set images if they don't exist
# # if not os.path.exists(os.path.join(val_path, 'ILSVRC2012_img_val.tar')):
# #     urllib.request.urlretrieve(url_images, os.path.join(val_path, 'ILSVRC2012_img_val.tar'))

# # Download the validation set labels if they don't exist
# if not os.path.exists(os.path.join(val_path, 'ILSVRC2012_validation_ground_truth.txt')):
#     urllib.request.urlretrieve(url_labels, os.path.join(val_path, 'ILSVRC2012_validation_ground_truth.txt'))
!ls

drive	     r101_1x_sk0_ema.pth  simclr_pytorch
r101_1x_sk0  sample_data	  SimCLRv2-Pytorch


In [ ]:
%cd "/content/drive/My Drive/computer vision project/"
!python simclr_pytorch/download.py r101_1x_sk0
!python simclr_pytorch/convert.py r101_1x_sk0/model.ckpt-250228 --ema
# !python SimCLRv2-Pytorch/verify.py r101_1x_sk0.pth

/content
/content/drive/My Drive/computer vision project
/content/drive/My Drive/computer vision project
  0% 0/5 [00:00<?, ?it/s]
  0% 0/1 [00:00<?, ?it/s]
 20% 1/5 [00:00<00:00,  7.21it/s]
  0% 0/71329 [00:00<?, ?it/s]
  2% 1073/71329 [00:00<00:06, 10720.45it/s]
  3% 2377/71329 [00:00<00:05, 12078.56it/s]
  6% 3985/71329 [00:00<00:04, 13893.36it/s]
  8% 5484/71329 [00:00<00:04, 14325.20it/s]
 10% 6917/71329 [00:00<00:04, 13929.43it/s]
 12% 8329/71329 [00:00<00:04, 13991.19it/s]
 14% 9730/71329 [00:00<00:04, 13814.83it/s]
 16% 11402/71329 [00:00<00:04, 14722.26it/s]
 18% 12877/71329 [00:00<00:04, 14257.93it/s]
 20% 14308/71329 [00:01<00:04, 13453.78it/s]
 22% 15664/71329 [00:01<00:04, 12821.76it/s]
 24% 17117/71329 [00:01<00:04, 13300.62it/s]
 26% 18472/71329 [00:01<00:03, 13371.48it/s]
 28% 20297/71329 [00:01<00:03, 14771.73it/s]
 31% 22014/71329 [00:01<00:03, 15473.40it/s]
 33% 23625/71329 [00:01<00:03, 15655.31it/s]
 36% 25417/71329 [00:01<00:02, 16163.08it/s]
 38% 27267/71329 [00:

In [ ]:
cities  = ["bloomington", "boulder", "buffalo", "calgary", "cambridge", "charlottesville", "columbus", "cupertino", "denver", "edmonton", "kitchener", "los_angeles", "montreal", "new_york", 'pittsburg', "san_francisco", "san_jose", "santa_monica", "seattle", "sioux_falls", "surrey", "vancouver", "washington_dc"]
%cd "/content/drive/My Drive/computer_vision_project/"
!ls
!for file in auto_arborist_cvpr2022_v0.8/tfrecords/*/*.zip; \
do echo $prefix; \
prefix=$(basename $file .zip); \
unzip -o $file -d data/$prefix; \
done
#!for file in auto_arborist_cvpr2022_v0.8/tfrecords/*/*.zip; \
#do prefix=$(basename $file .zip); \
#mkdir data/$prefix; \
#unzip $file -d /data/$prefix; \
#echo $prefix; \
#done
  

/content/drive/My Drive/computer_vision_project
 auto_arborist_cvpr2022_v0.8   ideas.gdoc	       r101_1x_sk0_ema.pth
 CV_project.ipynb	      'Important links.gdoc'  'Schedule .gdoc'
 data			       r101_1x_sk0	       simclr_pytorch

Archive:  auto_arborist_cvpr2022_v0.8/tfrecords/bloomington/bloomington.zip
  inflating: data/bloomington/test-00000-of-00025  
  inflating: data/bloomington/test-00001-of-00025  
  inflating: data/bloomington/test-00002-of-00025  
  inflating: data/bloomington/test-00003-of-00025  
  inflating: data/bloomington/test-00004-of-00025  
  inflating: data/bloomington/test-00005-of-00025  
  inflating: data/bloomington/test-00006-of-00025  
  inflating: data/bloomington/test-00007-of-00025  
  inflating: data/bloomington/test-00008-of-00025  
  inflating: data/bloomington/test-00009-of-00025  
  inflating: data/bloomington/test-00010-of-00025  
  inflating: data/bloomington/test-00011-of-00025  
  inflating: data/bloomington/test-00012-of-00025  
  inflating: dat

In [ ]:
%cd "/content/drive/My Drive/computer vision project/"
!pwd
from simclr_pytorch.resnet import get_resnet, name_to_params

pth_path = "/content/drive/My Drive/computer vision project/r101_1x_sk0_ema.pth"
data_dir = "/content/drive/MyDrive/computer vision project/data/bloomington"

model, _ = get_resnet(*name_to_params(pth_path))
model.load_state_dict(torch.load(pth_path)['resnet'])
model = model.to(device)

/content/drive/My Drive/computer vision project
/content/drive/My Drive/computer vision project
Archive:  /content/drive/My Drive/computer vision project/auto_arborist_cvpr2022_v0.8/tfrecords/bloomington/bloomington.zip
  inflating: /content/drive/My Drive/computer vision project/data/bloomington/test-00000-of-00025  
  inflating: /content/drive/My Drive/computer vision project/data/bloomington/test-00001-of-00025  
  inflating: /content/drive/My Drive/computer vision project/data/bloomington/test-00002-of-00025  
  inflating: /content/drive/My Drive/computer vision project/data/bloomington/test-00003-of-00025  
  inflating: /content/drive/My Drive/computer vision project/data/bloomington/test-00004-of-00025  
  inflating: /content/drive/My Drive/computer vision project/data/bloomington/test-00005-of-00025  
  inflating: /content/drive/My Drive/computer vision project/data/bloomington/test-00006-of-00025  
  inflating: /content/drive/My Drive/computer vision project/data/bloomington/te

In [ ]:
print(model)

ResNet(
  (net): Sequential(
    (0): Stem(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNormRelu(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU()
      )
      (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (1): Blocks(
      (blocks): ModuleList(
        (0): BottleneckBlock(
          (projection): Projection(
            (shortcut): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNormRelu(
              (0): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (1): Identity()
            )
          )
          (net): Sequential(
            (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNormRelu(
              (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
features = {
      'tree/city':
          tf.io.FixedLenFeature((), tf.string),
      'tree/id':
          tf.io.FixedLenFeature((), tf.string),
      'tree/idx':
          tf.io.FixedLenFeature((), tf.string),
      'tree/genus/label':
          tf.io.FixedLenFeature((), tf.int64),
      'tree/genus/genus':
          tf.io.FixedLenFeature((), tf.string),
      'tree/latitude':
          tf.io.FixedLenFeature((), tf.float32),
      'tree/longitude':
          tf.io.FixedLenFeature((), tf.float32),
      'aerial/encoded': tf.io.FixedLenFeature((), tf.string),
      'streetlevel/encoded': tf.io.FixedLenFeature((), tf.string),
  }


In [ ]:
# type: "train" or "test"
def get_data(type, city):
  pth = "/content/drive/My Drive/computer_vision_project/data/"+city+"/"
  images = []
  labels = []
  for i in range(25):
    str_i = str(i) if i >= 10 else "0"+str(i)
    filename = f'{type}-000{str_i}-of-00025'
    path = pth+filename
    print(path)
    raw_dataset = tf.data.TFRecordDataset(path)
    for record in raw_dataset:
      example = tf.io.parse_single_example(record, features)
      sv_image = tf.image.decode_jpeg(example['streetlevel/encoded']).numpy()
      label_id = int(example['tree/genus/label'].numpy())
      images.append(sv_image)
      labels.append(label_id)
  images = torch.tensor(images, device="cuda")
  labels = torch.tensor(labels, device="cuda")
  return images, labels



In [ ]:
%ls
images, labels = get_data("train", "bloomington")
print(images.shape, labels.shape)

drive/  sample_data/
/content/drive/My Drive/computer_vision_project/data/bloomington/train-00000-of-00025
/content/drive/My Drive/computer_vision_project/data/bloomington/train-00001-of-00025
/content/drive/My Drive/computer_vision_project/data/bloomington/train-00002-of-00025
/content/drive/My Drive/computer_vision_project/data/bloomington/train-00003-of-00025
/content/drive/My Drive/computer_vision_project/data/bloomington/train-00004-of-00025
/content/drive/My Drive/computer_vision_project/data/bloomington/train-00005-of-00025
/content/drive/My Drive/computer_vision_project/data/bloomington/train-00006-of-00025
/content/drive/My Drive/computer_vision_project/data/bloomington/train-00007-of-00025
/content/drive/My Drive/computer_vision_project/data/bloomington/train-00008-of-00025
/content/drive/My Drive/computer_vision_project/data/bloomington/train-00009-of-00025
/content/drive/My Drive/computer_vision_project/data/bloomington/train-00010-of-00025
/content/drive/My Drive/computer_